# RAG with a Tool Use approach

In this notebook, we'll look at how we can implement RAG use cases using a tool use approach.

Tool use allows for greater flexibility of accessing data sources, thus unlocking new use cases not possible with a standard RAG approach.

In an enterprise setting with diverse data sources with non-homogeneous formats (structured/semi-structured/unstructured), this approach becomes even more useful.

We'll look at a few example use cases:
- Tool selection
- Multi-step searches
- Structured queries
- Structured data queries
- Action - plotting charts

In [ ]:
import os

import cohere

co = cohere.Client(os.getenv("COHERE_API_KEY"))

from tool_def import (analyze_evaluation_results, search_code_examples,
                      search_company_information, search_developer_docs, tools)

In [ ]:
functions_map = {
    "search_code_examples": search_code_examples,
    "search_developer_docs": search_developer_docs,
    "search_company_information": search_company_information,
    "analyze_evaluation_results": analyze_evaluation_results,
}

In [ ]:
preamble = """## Task & Context
You are an assistant who helps developers use Weights & Biases. The company is also referred to as Wandb or W&B for short. You are equipped with a number of tools that can provide different types of information. If you can't find the information you need from one tool, you should try other tools if there is a possibility that they could provide the information you need."""

# Function to run the assistant over multiple chat turns

In [ ]:
model = "command-r-plus"


def run_assistant(message, chat_history=[], show_documents=False):
    # Step 1: Get user message
    print(f"Question:\n{message}")
    print("=" * 50)

    # Step 2: Generate tool calls (if any)
    response = co.chat(
        message=message,
        model=model,
        preamble=preamble,
        tools=tools,
        chat_history=chat_history,
    )

    while response.tool_calls:
        tool_calls = response.tool_calls

        if response.text:
            print("Intermediate response:")
            print(response.text, "\n")
        print("Tool calls:")
        for call in tool_calls:
            print(f"Tool name: {call.name} | Parameters: {call.parameters}")
        print("=" * 50)

        # Step 3: Get tool results
        tool_results = []
        for tc in tool_calls:
            tool_call = {"name": tc.name, "parameters": tc.parameters}
            tool_output = functions_map[tc.name](**tc.parameters)
            tool_results.append({"call": tool_call, "outputs": [tool_output]})

        # Step 4: Generate response and citations
        response = co.chat(
            message="",
            model=model,
            preamble=preamble,
            tools=tools,
            tool_results=tool_results,
            chat_history=response.chat_history,
        )

        # Append the current chat turn to the chat history
        chat_history = response.chat_history

    # Print final response
    print("Final response:")
    print(response.text)
    print("=" * 50)

    # Print citations (if any)
    if response.citations:
        print("Citations:")
        for citation in response.citations:
            print(citation)
        if show_documents:
            print("\nCited Documents:")
            for document in response.documents:
                print(document)
        print("=" * 50)

    return chat_history

# Tool selection

In [ ]:
chat_history = run_assistant("Where can I find the output of a run")
# Chooses search_developer_docs

In [ ]:
chat_history = run_assistant("Where are Wandb's offices")
# Chooses search_company_information

# Multi-step searches

In [ ]:
chat_history = run_assistant(
    "What's that feature to automate hyperparameter search? Do you have some code examples?"
)
# Does two steps of tool use
# Returns two code examples - Selecting Hyperparameters with Sweeps (Keras) & Create a hyperparameter search with W&B PyTorch integration

# Structured queries

In [ ]:
chat_history = run_assistant("Any jupyter notebook for Data Versioning with Artifacts?")
# Searches for file_type = ipynb

In [ ]:
chat_history = run_assistant("Any code examples for data versioning with artifacts?")
# Doesn't need to specify file_type as it's a generic question

In [ ]:
# English: Code examples on how to visualize datasets
chat_history = run_assistant("データセットを視覚化する方法のコード例?")
# Searches for language = ja
# Returns a code example in Japanese about tables

# Structured data queries

In [ ]:
chat_history = run_assistant("What's the average evaluation score in run A")
# Answer: 0.63

In [ ]:
chat_history = run_assistant(
    "What's the latency of the highest-scoring run for the summarize_article use case?"
)
# Answer: 4.8

In [ ]:
chat_history = run_assistant(
    "Which use case uses the least amount of tokens on average? Show the comparison in a table."
)
# Answer: extract_names (106.25), draft_email (245.75), summarize_article (355.75)

# Action - plotting charts

In [ ]:
chat_history = run_assistant(
    "Create a plot of the average evaluation score for each temperature setting for the extract_names use case."
)
# Answer: temp 0.3 (0.46 avg score) vs temp 0.5 (0.7 avg score). And draws a plot